In [2]:
from mxnet import autograd, nd
from mxnet.gluon import nn

def corr2d(X, K):
    h, w = K.shape
    Y = nd.zeros((X.shape[0]-h + 1, X.shape[1] -w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i: i+h, j:j+w]*K).sum()
    return Y


In [3]:
X = nd.array([[0,1,2],[3,4,5],[6,7,8]])
K = nd.array([[0,1],[2,3]])
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

In [4]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1, ))
    
    def forward(self, x):
        return corr2d(x, self.weight.data())+self.bias.data()
    
# 卷积层

In [5]:
# 检测边缘 卷积层的一个小应用 构造边缘
X = nd.ones((6,8))
X[:, 2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [6]:
K = nd.array([[1, -1]])
# 构造内核  相邻元素相同则为0 ， 相邻元素不相同不是0

In [7]:
Y = corr2d(X, K)
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [8]:
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()

In [9]:
X = X.reshape(1,1,6,8)
Y = Y.reshape(1,1,6,7)

for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) **2
    l.backward()
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i+1) % 2 == 0:
        print("batch %d, loss %.3f" % (i+1, l.sum().asscalar()))

batch 2, loss 4.949
batch 4, loss 0.831
batch 6, loss 0.140
batch 8, loss 0.024
batch 10, loss 0.004


In [10]:
conv2d.weight.data().reshape((1,2))


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>

In [11]:
def comp_conv2d(conv2d, X):
    conv2d.initialize()
    X = X.reshape((1,1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2D(1, kernel_size=3, padding=1)
X =nd.random.uniform(shape=(8 , 8))
comp_conv2d(conv2d, X).shape

(8, 8)

In [12]:
conv2d = nn.Conv2D(1 ,kernel_size=3 , padding=1, strides=2)
comp_conv2d(conv2d, X).shape

(4, 4)

In [13]:
conv2d = nn.Conv2D(1, kernel_size=(3, 5), padding=(0, 1), strides=(3, 4))
comp_conv2d(conv2d, X).shape

(2, 2)